In [1]:
import faiss
import numpy as np
import pickle
import json
from conf import *

In [2]:
with open(rid_output_path, 'rb') as f:
    rid_keys, rid_values = pickle.load(f)

In [3]:
with open(sid_output_path, 'rb') as f:
    sid_keys, sid_values = pickle.load(f)

In [4]:
index = faiss.IndexFlatIP(emb_size)
# index.add(item_embeddings)
index_with_id = faiss.IndexIDMap(index)
index_with_id.add_with_ids(rid_values, rid_keys)

In [19]:
import redis
work10redis = redis.StrictRedis(host="sg-prod-research-worker-10", port=6379)


In [23]:
from collections import Counter
redisPrefix = "live_node2vec_"
redisTTL = 86400*7

In [24]:
pipeline = work10redis.pipeline(transaction=False)
D, I = index_with_id.search(rid_values[:10], top_k)
print("rid diversity", len(Counter(I.reshape(-1))))
for i in range(len(rid_keys[:10])):
    rid = rid_keys[i]       
    redisKey = redisPrefix + "r2r_" + str(rid)
    sim_res = dict(zip(I[i].tolist(), D[i].tolist()))
    redisVal = json.dumps(sim_res)
    print(redisKey, redisVal)
    pipeline.set(redisKey, redisVal)
    pipeline.expire(redisKey, redisTTL)

D, I = index_with_id.search(sid_values[:10], top_k)
print("sid diversity", len(Counter(I.reshape(-1))))
for i in range(len(sid_keys[:10])):
    sid = sid_keys[i]       
    redisKey = redisPrefix + "s2r_" + str(sid)
    sim_res = dict(zip(I[i].tolist(), D[i].tolist()))
    redisVal = json.dumps(sim_res)
    print(redisKey, redisVal)
    pipeline.set(redisKey, redisVal)
    pipeline.expire(redisKey, redisTTL)

pipeline.execute()

rid diversity 451
live_node2vec_r2r_12103424001423753 {"12103424001423753": 11.837394714355469, "3940649678214774": 9.575607299804688, "7881299351192689": 9.27463436126709, "3377699723584550": 9.128829956054688, "6192448700875200": 8.991686820983887, "281474984273511": 8.875484466552734, "12384898982478601": 8.699792861938477, "12666373952126650": 8.635327339172363, "3940649679242682": 8.284944534301758, "8162774329042156": 8.283961296081543, "3659174700309816": 8.19422435760498, "10133099163619027": 8.077488899230957, "6192448705054283": 8.034442901611328, "2814749773265526": 7.980076313018799, "10133099167197067": 7.971652984619141, "3659174699902123": 7.9503631591796875, "281474981332485": 7.945237159729004, "10977524092311262": 7.925320625305176, "8162774330745689": 7.920438289642334, "4222124654933741": 7.899523735046387, "281474980518414": 7.85465145111084, "3377699723768120": 7.841625213623047, "844424933888788": 7.831695079803467, "8162774329677267": 7.824056625366211, "4222124

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [2]:
with open(output_path, 'rb') as f:
    item_embeddings, iid_map_reverse = pickle.load(f)
    
item_embeddings.shape, len(iid_map_reverse)

((63014, 64), 63014)

In [3]:
iids = np.array(list(iid_map_reverse.values()))
index = faiss.IndexFlatIP(emb_size)
# index.add(item_embeddings)
index_with_id = faiss.IndexIDMap(index)
index_with_id.add_with_ids(item_embeddings, iids)

In [15]:
%%time
D, I = index.search(item_embeddings, top_k)

CPU times: user 2min 16s, sys: 8.21 s, total: 2min 24s
Wall time: 3.36 s


In [107]:
redisPrefix = "live_i2i_recall_"
D, I = index_with_id.search(item_embeddings[:10], top_k)
for i in range(len(iids[:10])):
    iid = iids[i]       
    redisKey = redisPrefix + str(iid)
    redisField = "pinsage"
    sim_res = dict(zip(I[i], D[i]))
#     redisVal = json.dumps(sim_res)
    print(redisKey, sim_res)
    break

live_i2i_recall_2814749772758281 {11540474051513042: 1.3184413, 1125899914095828: 1.2865142, 12666373960154673: 1.2766081, 8162774327650154: 1.2765301, 1688849867535072: 1.2719337, 12384898977104817: 1.2632844, 10696049116478908: 1.2630621, 1970324843639407: 1.260058, 12103423999491613: 1.2560008, 2251799819466443: 1.2548497, 8725724284849772: 1.2547345, 8162774330039799: 1.253662, 7318349398643511: 1.2399933, 10414574145189099: 1.2385142, 10133099171089580: 1.2366489, 1125899908731509: 1.235729, 11540474046582856: 1.229206, 10133099169775605: 1.2284209, 7881299351950049: 1.227714, 1970324842067134: 1.227393, 10133099164982367: 1.2240038, 2533274797975868: 1.2234502, 1688849867516167: 1.2225288, 3377699727060835: 1.2217157, 3659174700583601: 1.2192916, 8162774327412037: 1.2191353, 10133099161906519: 1.2189711, 7318349400388165: 1.2183814, 10977524093062633: 1.2170471, 4222124652201866: 1.2145046, 7036874421018720: 1.2125465, 10696049116990144: 1.2116399, 8725724285568101: 1.211271, 703

In [7]:
%%time
D, I = index_with_id.search(item_embeddings, top_k)

CPU times: user 5min 12s, sys: 15.5 s, total: 5min 28s
Wall time: 7.07 s


In [19]:
I[0]

array([    0,   113,     7,  1839,   533,    33,   190,   282,    79,
          12,  5950,    41,   799,   227,    66,   170,   377,   309,
         441,   315,   127,   329,   291,   375,   601, 11155, 22927,
        1223,   459,  6337,  7435,   385,   502,   512,  1469,   790,
         373,   842,   538,   182,   426, 34977,  1761,   104,   333,
         151,  1112,    76,  7990,  8134])

In [20]:
I[1]

array([    0,     7,   282,   113,   533,    79,   375,    33,  1839,
         441,   190,   227,    66,   329,  1223,   638,    12,   385,
         373,  6337,  7355, 40642,   842,  3142,   170, 10307,   291,
       59853,   459,   799,   377,   315,  5950,   127, 17253, 11155,
         601,   505,    67,   538,   151,   231,  6903,   462,   408,
         790,  1177, 22927,   182,   398])

In [23]:
len(set(I[100]) & set(I[1005]))

21

In [21]:
I[100]

array([ 2772,  6337,   138, 42441,    63,   227,    79,     0,  3142,
         329,    66,    33,   533, 33277,    67, 59853,   638,  4650,
       13127,  1223,     7,   601, 22384,   144,  1529,   261,   398,
         125,   231,   385,  8109,  8134, 47614,   282,  7355,   171,
       26744, 25616,  1198,   377, 17418,   307,   113,   842,   459,
       14897,  4767, 50351, 16444,   375])

In [22]:
I[102]

array([ 6337,     0,   282,   533,   227,     7,    33,    79,   113,
         375,   385,  1529,   601,   170,   231,   329,  1839,  2772,
       59853,   638,   398,   799,  2239,   190,   459,  3298,   377,
         144,  5950,  1425, 10307,   538, 11155,  1223, 20030,   138,
        7355,   104,  2762,    66,   124,   426,  9415,    27,   373,
         743,   417,   411,   742,  3142])

In [18]:
from collections import Counter
Counter(I.reshape(-1))

Counter({0: 19664,
         113: 14131,
         7: 15917,
         1839: 12701,
         533: 15620,
         33: 11329,
         190: 9807,
         282: 13363,
         79: 12543,
         12: 8384,
         5950: 7798,
         41: 6698,
         799: 10478,
         227: 10534,
         66: 8681,
         170: 6272,
         377: 5126,
         309: 5475,
         441: 7836,
         315: 4304,
         127: 4736,
         329: 9529,
         291: 5011,
         375: 9402,
         601: 6776,
         11155: 3769,
         22927: 3104,
         1223: 8935,
         459: 5096,
         6337: 11251,
         7435: 4789,
         385: 8091,
         502: 2752,
         512: 2229,
         1469: 6456,
         790: 4148,
         373: 4818,
         842: 6377,
         538: 3373,
         182: 2495,
         426: 4041,
         34977: 1873,
         1761: 1632,
         104: 3427,
         333: 677,
         151: 1047,
         1112: 1667,
         76: 1466,
         7990: 1150,
     

In [17]:
len(Counter(I.reshape(-1)))

37311

In [13]:
sorted(dict(Counter(I.reshape(-1))).items(), key=lambda x:x[1], reverse=True)

[(562949960780104, 44016),
 (7881299353512452, 41715),
 (4222124653830614, 41287),
 (7318349398643511, 40517),
 (2533274797516156, 40280),
 (562949956946375, 36818),
 (10414574138755797, 34570),
 (4785074602357299, 34569),
 (3096224744087574, 34072),
 (7318349401870139, 33549),
 (12103424002204530, 33439),
 (4222124653133291, 32248),
 (4222124655509426, 31106),
 (11821949026017708, 30569),
 (12947848931173242, 29955),
 (7318349396449067, 29714),
 (12666373957062263, 28770),
 (8162774327436505, 28324),
 (3940649681059368, 28267),
 (8162774324630810, 28083),
 (8725724285372824, 27972),
 (281474983904778, 27834),
 (11821949026798721, 27196),
 (2251799821031256, 26858),
 (4222124653669437, 26821),
 (3096224747460085, 26458),
 (4222124653699087, 26369),
 (6755399376294689, 26038),
 (10414574145464266, 25732),
 (12103424005434629, 25584),
 (3940649679870532, 25277),
 (3377699726905567, 25032),
 (3940649680678481, 24673),
 (10977524099641765, 24571),
 (6755399380319013, 24519),
 (118219490272